In [115]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

import os
import subprocess
import shutil
from glob import glob

In [2]:
filein = 'sample_data/visitas_1_2022.xlsx'
database_file = 'sample_data/listado_pacientes.xlsx'

In [3]:
df_visits = pd.read_excel(filein)
df_visits.head()

,Fecha,Profesional,Paciente,DNI,Concepto,Euros
0,2022-04-07,Elena,Claudio Casemiro Herranz Echeverría,20121384H,terapia,57
1,2022-04-07,Elena,Claudio Casemiro Herranz Echeverría,20121384H,terapia,80
2,2022-04-30,Elena,Claudio Casemiro Herranz Echeverría,20121384H,terapia,71
3,2022-04-04,Elena,Claudio Casemiro Herranz Echeverría,20121384H,terapia,72
4,2022-04-16,Elena,Claudio Casemiro Herranz Echeverría,20121384H,terapia,70


In [4]:
df_patients = pd.read_excel(database_file)
df_patients

,Nombre,Fecha nacimiento,Padre/Madre (si menor),DNI,Telefono 1,Telefono 2,Direccion,Mail,Profesional,Tipo de tratamiento,Inicio tratamiento,Notas
0,Lucila Lourdes Aller Lillo,1952-02-22,NaN,14665734Z,699234346,NaN,"Glorieta de Anselma Guitart 519\nSoria, 07099",ciprianomolina@yahoo.com,Psicologo3,terapia,2013-06-21,NaN
1,Rosendo Iglesias,1990-09-21,NaN,56683053J,644268972,645133057.0,Urbanización de José Ángel Morata 54\nSanta Cr...,bertapombo@gmail.com,Oriol,terapia,1985-10-26,Actor
2,Claudio Casemiro Herranz Echeverría,1975-06-20,NaN,20121384H,626127369,635817854.0,"Ronda Salud Girona 75 Apt. 46 \nLa Rioja, 49676",wdieguez@hotmail.com,Elena,terapia,1991-02-08,Técnico en ciencias biológicas
3,Rosario del Sarmiento,2007-05-31,Wilfredo del Gomila,70491670M,690722981,NaN,"Cañada de Gisela Beltran 93\nCiudad, 17934",lermaclaudia@yahoo.com,Elena,terapia,1973-12-20,NaN
4,Chucho Pallarès-Patiño,1974-07-10,NaN,75675820R,665470525,NaN,"Cuesta Amaya Pintor 25 Piso 3 \nAlicante, 45174",pedrazagabino@hotmail.com,Oriol,terapia,1998-09-16,NaN
5,Vidal Barros Lillo,1919-11-17,NaN,79783301G,682931771,674538833.0,Rambla de Nereida Tejedor 562 Puerta 7 \nHuelv...,ividal@gmail.com,Psicologo3,terapia,1923-03-11,Moldeador y machero
6,Nicanor Ojeda Haro,1938-02-09,NaN,16548172V,614388117,NaN,"Avenida Quique Cepeda 4 Puerta 9 \nTarragona, ...",jesus67@yahoo.com,Oriol,terapia,1967-10-23,NaN
7,Marcos Borja González,2011-07-12,Aitor del Mir,60196888T,616527111,679116126.0,Calle Epifanio Ibañez 594 Puerta 3 \nLa Coruña...,ncastello@gmail.com,Elena,terapia,1966-06-30,Personal de limpieza
8,Josep del Mosquera,1922-03-05,NaN,83974519R,600169101,NaN,Vial de Juan Carlos Calderon 19 Piso 8 \nSanta...,carnerotelmo@yahoo.com,Elena,terapia,1932-04-30,NaN
9,Lucila Elodia Cabo Vigil,2015-02-07,Bienvenida de Briones,77018110B,688132038,NaN,"Ronda Celestino Jover 263 Piso 4 \nSegovia, 20778",edmundopalma@hotmail.com,Elena,terapia,1906-09-25,NaN


In [5]:
output_path = 'facturas/'

In [6]:
os.makedirs(output_path, exist_ok=True)

In [7]:
template_file = 'template/plantilla_factura.tex'

In [8]:
with open(template_file) as fd:
    template = fd.read()

In [9]:
params = {'NOMFACTURA' : 'test name factura',
          'DNI' : 'test dni',
          'DIRECCIO' : 'test address',
          'NUMEROFACTURA' : 'test numero factura',
          'NOMPACIENT' : 'test nombre paciente',
          'MES' : 'test mes',
          'ANY' : 'test año',
          'NUMEROSESSIONS' : 'test numero sessions',
          'PREUTOTAL' : 'test preu total',
          'MES2' : 'test mes 2',
          'ANY2' : 'test año 2'}

In [10]:
def replace_params_in_template(template, params):
    for key, value in params.items():
        template = template.replace(f'{{{key}}}', f'{value}')
    return template

In [156]:
def month_to_spanish_name(month):
    meses = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']
    return meses[month-1]

In [161]:
def extract_month_and_year(df_visits):
    month = df_visits['Fecha'].apply(lambda x: x.month).mode().values[0]
    year  = df_visits['Fecha'].apply(lambda x: x.year) .mode().values[0]
    return month, year

In [162]:
def parse_visit(row):
    date_parsed = row['Fecha'].strftime('%d/%m/%Y')
    line = '&{} &{} \EURtm \\\\'.format(date_parsed, row['Euros'])
    return line

In [184]:
def process_patient_visits(dni, df_visits, patient, index):
    period_month, period_year = extract_month_and_year(df_visits)

    invoice_date = datetime(period_year, period_month, 1) + timedelta(days=40)
    invoice_number = f'{period_year}/{period_month}/{index}'

    padres = patient['Padre/Madre (si menor)']
    nom_factura = padres if isinstance(padres, str) else patient['Nombre']
    
    visits_parsed = group.sort_values('Fecha').apply(parse_visit, axis=1)
    visits_str = '\n'.join(visits_parsed.values)

    n_sessions = df_visits.shape[0]
    sessions = 'sesiones' if n_sessions > 1 else 'sesión'

    params = {'NOMFACTURA' : nom_factura,
              'DNI' : dni,
              'DIRECCIO' : patient['Direccion'],
              'NUMEROFACTURA' : invoice_number,
              'NOMPACIENT' : patient['Nombre'],
              'MES' : month_to_spanish_name(period_month),
              'ANY' : period_year,
              'VISITS' : visits_str,
              'NUMEROSESSIONS' : f'{n_sessions} {sessions}',
              'PREUTOTAL' : df_visits['Euros'].sum(),
              'MES2' : month_to_spanish_name(invoice_date.month),
              'ANY2' : invoice_date.year}
    
    return params

In [185]:
month, year = extract_month_and_year(df_visits)
temp_fname  = f'{year}_{month:02}'

In [186]:
temp_dir = os.path.join(output_path, 'temp', temp_fname)
os.makedirs(temp_dir, exist_ok=True)

In [187]:
template_files = glob('template/*png')
[shutil.copy(fname, temp_dir) for fname in template_files]

['facturas/temp/2022_04/logo.png', 'facturas/temp/2022_04/firma.png']

In [192]:
groups = df_visits.groupby('DNI')

for i, (dni, group) in enumerate(groups):
    print(dni)
    patient = df_patients[df_patients['DNI'] == dni].iloc[0] # TODO: Detect if no patient found!
    params = process_patient_visits(dni, group, patient, i+1)
    print(params)
    
    tex_file = replace_params_in_template(template, params)
    fname = f'{dni}.tex'
    fileout = os.path.join(temp_dir, fname)
    
    with open(fileout, 'w') as fd:
        fd.write(tex_file)
        
    cmd_result = subprocess.run([f"cd {temp_dir}; pdflatex {fname}"], capture_output=True, text=True, shell=True)
    if cmd_result.returncode != 0:
        print(f'Error in file {fileout}')
    # cmd_result.stdout

16548172V
{'NOMFACTURA': 'Nicanor Ojeda Haro', 'DNI': '16548172V', 'DIRECCIO': 'Avenida Quique Cepeda 4 Puerta 9 \nTarragona, 48153', 'NUMEROFACTURA': '2022/4/1', 'NOMPACIENT': 'Nicanor Ojeda Haro', 'MES': 'abril', 'ANY': 2022, 'VISITS': '&06/04/2022 &69 \\EURtm \\\\\n&06/04/2022 &65 \\EURtm \\\\\n&07/04/2022 &52 \\EURtm \\\\\n&09/04/2022 &72 \\EURtm \\\\\n&12/04/2022 &65 \\EURtm \\\\\n&13/04/2022 &78 \\EURtm \\\\\n&21/04/2022 &75 \\EURtm \\\\\n&21/04/2022 &74 \\EURtm \\\\', 'NUMEROSESSIONS': '8 sesiones', 'PREUTOTAL': 550, 'MES2': 'mayo', 'ANY2': 2022}
20121384H
{'NOMFACTURA': 'Claudio Casemiro Herranz Echeverría', 'DNI': '20121384H', 'DIRECCIO': 'Ronda Salud Girona 75 Apt. 46 \nLa Rioja, 49676', 'NUMEROFACTURA': '2022/4/2', 'NOMPACIENT': 'Claudio Casemiro Herranz Echeverría', 'MES': 'abril', 'ANY': 2022, 'VISITS': '&04/04/2022 &72 \\EURtm \\\\\n&05/04/2022 &76 \\EURtm \\\\\n&07/04/2022 &57 \\EURtm \\\\\n&07/04/2022 &80 \\EURtm \\\\\n&16/04/2022 &70 \\EURtm \\\\\n&28/04/2022 &79 \\EUR

In [193]:
from zipfile import ZipFile

In [194]:
invoices = glob(os.path.join(temp_dir, '*pdf'))

In [195]:
with ZipFile('sample.zip', 'w') as fd_zip:
    for invoice in invoices:
        fd_zip.write(invoice, os.path.basename(invoice))